## Conducting Walk-Forward Optimization with VectorBT

In [1]:
import numpy as np
import scipy.stats as stats
import vectorbt as vbt
from IPython.display import Markdown, display

Define the start and end dates for data download

In [2]:
start = "2016-01-01 UTC"
end = "2020-01-01 UTC"

Download historical closing prices for the symbol "AAPL" from Yahoo Finance

In [3]:
prices = vbt.YFData.download("AAPL", start=start, end=end).get("Close")

Split the prices data into in-sample and out-of-sample sets

In [4]:
(in_price, in_indexes), (out_price, out_indexes) = prices.vbt.rolling_split(
    n=30,
    window_len=365 * 2,
    set_lens=(180,),
    left_to_right=False,
)

Function to simulate all parameter combinations and calculate Sharpe ratios

In [5]:
def simulate_all_params(price, windows, **kwargs):
    fast_ma, slow_ma = vbt.MA.run_combs(
        price, windows, r=2, short_names=["fast", "slow"]
    )
    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)
    pf = vbt.Portfolio.from_signals(price, entries, exits, **kwargs)
    return pf.sharpe_ratio()

Function to get the best index based on performance

In [6]:
def get_best_index(performance):
    return performance[performance.groupby("split_idx").idxmax()].index

Function to get the best parameters from the best index

In [7]:
def get_best_params(best_index, level_name):
    return best_index.get_level_values(level_name).to_numpy()

Function to simulate the best parameters and calculate Sharpe ratios

In [8]:
def simulate_best_params(price, best_fast_windows, best_slow_windows, **kwargs):
    fast_ma = vbt.MA.run(price, window=best_fast_windows, per_column=True)
    slow_ma = vbt.MA.run(price, window=best_slow_windows, per_column=True)

    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)

    pf = vbt.Portfolio.from_signals(price, entries, exits, **kwargs)
    return pf.sharpe_ratio()

Define the range of windows for moving averages

In [9]:
windows = np.arange(10, 40)

Simulate all parameter combinations for in-sample data and calculate Sharpe ratios

In [10]:
in_sharpe = simulate_all_params(in_price, windows, direction="both", freq="d")

Get the best index and parameters from the in-sample Sharpe ratios

In [11]:
in_best_index = get_best_index(in_sharpe)

In [12]:
in_best_fast_windows = get_best_params(in_best_index, "fast_window")

In [13]:
in_best_slow_windows = get_best_params(in_best_index, "slow_window")

In [14]:
in_best_window_pairs = np.array(list(zip(in_best_fast_windows, in_best_slow_windows)))

Simulate the best parameters for out-of-sample data and calculate Sharpe ratios

In [15]:
out_test_sharpe = simulate_best_params(
    out_price, in_best_fast_windows, in_best_slow_windows, direction="both", freq="d"
)

In [16]:
display(out_test_sharpe)

ma_window  ma_window  split_idx
10         11         0            0.104948
12         13         1            0.318327
                      2            0.971232
10         11         3            1.386777
12         13         4            1.303281
10         11         5            2.133288
                      6            2.043516
18         23         7            1.756907
                      8            2.219373
                      9            2.283883
                      10           2.543999
                      11           2.724649
                      12           2.389901
                      13           2.838679
                      14           2.393304
                      15           1.116309
23         26         16           0.670555
                      17           0.594145
                      18           0.816461
24         25         19           0.276339
                      20          -0.052507
                      21          -0.363493


Perform a t-test to compare the in-sample and out-of-sample Sharpe ratios

In [17]:
in_sample_best = in_sharpe[in_best_index].values
out_sample_test = out_test_sharpe.values

In [18]:
t, p = stats.ttest_ind(a=out_sample_test, b=in_sample_best, alternative="greater")

In [19]:
display(t, p)

-1.0849398748573142

0.8587812940925162

In [20]:
display(out_test_sharpe)

ma_window  ma_window  split_idx
10         11         0            0.104948
12         13         1            0.318327
                      2            0.971232
10         11         3            1.386777
12         13         4            1.303281
10         11         5            2.133288
                      6            2.043516
18         23         7            1.756907
                      8            2.219373
                      9            2.283883
                      10           2.543999
                      11           2.724649
                      12           2.389901
                      13           2.838679
                      14           2.393304
                      15           1.116309
23         26         16           0.670555
                      17           0.594145
                      18           0.816461
24         25         19           0.276339
                      20          -0.052507
                      21          -0.363493


Defines the alternative hypothesis. The following options are available (default is ‘two-sided’):
*  ‘two-sided’: the means of the distributions underlying the samples are unequal.
*  ‘less’: the mean of the distribution underlying the first sample is less than the mean of the distribution underlying the second sample.
*  ‘greater’: the mean of the distribution underlying the first sample is greater than the mean of the distribution underlying the second sample.

**Jason Strimpel** is the founder of <a href='https://pyquantnews.com/'>PyQuant News</a> and co-founder of <a href='https://www.tradeblotter.io/'>Trade Blotter</a>. His career in algorithmic trading spans 20+ years. He previously traded for a Chicago-based hedge fund, was a risk manager at JPMorgan, and managed production risk technology for an energy derivatives trading firm in London. In Singapore, he served as APAC CIO for an agricultural trading firm and built the data science team for a global metals trading firm. Jason holds degrees in Finance and Economics and a Master's in Quantitative Finance from the Illinois Institute of Technology. His career spans America, Europe, and Asia. He shares his expertise through the <a href='https://pyquantnews.com/subscribe-to-the-pyquant-newsletter/'>PyQuant Newsletter</a>, social media, and has taught over 1,000+ algorithmic trading with Python in his popular course **<a href='https://gettingstartedwithpythonforquantfinance.com/'>Getting Started With Python for Quant Finance</a>**. All code is for educational purposes only. Nothing provided here is financial advise. Use at your own risk.